In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np
import os
import sys
import json
import webbrowser
from json.decoder import JSONDecodeError
import os
import io
from six.moves.urllib.request import urlopen
import numpy as np
import librosa
import librosa.display
import soundfile as sf
import glob
import pickle
import config

import pydub
from urllib.request import urlopen
import requests
from shutil import copyfileobj
from tempfile import NamedTemporaryFile
from urllib.request import urlopen, Request
import matplotlib.pyplot as plt

## Connecting to spotify and obtaining song info

Just going by the spotify playlist criteria was a bad idea- turns out techno playlists dont always have techno songs in them. Same goes for a lot of other genres. Therefore will have to look for the genre of the song specifically as well.

In [2]:
#you will need your own client id and secret
client_id= config.client_id
client_secret =config.client_secret

In [3]:
#this code block is needed to connect to spotify 
username = '1143043561'
auth = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)


try:
    token = auth.get_access_token()
except:
    os.remove(f'.cache-{username}')
    token = auth.get_access_token()
    
#create spotify object
spotify = spotipy.Spotify(auth=token)

In [69]:
def get_songs_from_playlist_search(limit=10,artist_name='techno'):
    '''takes a search term such as techno, finds playlists with techno
        in the name and then returns information about the songs in
        the playlist'''
    playlist_ids=[]
    song_ids = []
    playlists =[]
    song_ids=[]
    preview_urls=[]
    song_names=[]
    
    count=0
    
    results = spotify.search(q=artist_name,limit=limit, type='playlist')
    #find playlists with specific search term
    for playlist_info in results['playlists']['items']:
        playlist_ids.append(playlist_info['id'])
    
    #add all the songs in the playlists to another list
    for playlist in playlist_ids:
        source_playlist=spotify.user_playlist(username,playlist)
        song_set=source_playlist['tracks']
        songs=song_set['items']
        
        while song_set['next']:
            song_set=spotify.next(song_set)
            for song in song_set['items']:
                songs.append(song)
        playlists.append(songs)
    
        
    #loop over the song lists and extract information
    for song_list in playlists:
        for track in song_list:
            
            try:
                if track['track']['id']==None:
                    pass
                else:
                    try:
                        song_ids.append(track['track']['id'])
                        song_names.append(track['track']['name'])
                        preview_urls.append(track['track']['preview_url'])

                    except:
                        print(count)
                        count+=1
            except:
                print(count)
                count+=1
    
    return (song_ids,song_names,preview_urls)

In [5]:
def get_genre(song_ids):
    #get genres using song ids
    genres=[]
    tracks=spotify.tracks(song_ids)
    artist_id=track['artists'][0]['id']
    album_id=track['album']['id']
    album_info=spotify.album(album_id)
    #first looks to see if album has genre and if not looks at artist genres
    genres=album_info['genres']
    if not genres:
        artist=spotify.artist(artist_id)
        genres=artist['genres']
    return genres

In [16]:
#going to put the limit a lot higher than what i want because only some of them will have genres i want
techno_id, techno_names, techno_urls = get_songs_from_playlist_search(limit=30)

In [60]:
techno_genres = []
for song in techno_id:
    try:
        techno_genres.append(get_genre(song))
    except:
        techno_genres.append(None)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...4secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [61]:
len(techno_genres)

4298

In [63]:
pop_id,pop_names,pop_urls= get_songs_from_playlist_search(limit=15,artist_name='pop')

In [65]:
pop_genres=[]
for song in pop_id:
    try:
        pop_genres.append(get_genre(song))
    except:
        pop_genres.append(None)
        print('no')
        

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs


In [70]:
grime_id,grime_names,grime_urls =get_songs_from_playlist_search(limit=17,artist_name='grime')

0


In [71]:
grime_genres=[]
for song in grime_id:
    try:
        grime_genres.append(get_genre(song))
    except:
        grime_genres.append(None)
        

retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...2secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...2secs
retrying ...2secs
retrying ...1secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...3secs
retrying ...1secs


In [74]:
garage_id,garage_names,garage_urls =get_songs_from_playlist_search(limit=13,artist_name='uk garage')

In [75]:
garage_genres=[]
for song in garage_id:
    try:
        garage_genres.append(get_genre(song))
    except:
        garage_genres.append(None)

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [76]:
hip_hop_id,hip_hop_names,hip_hop_urls=get_songs_from_playlist_search(limit=15,artist_name='hip hop')

0
1
2


In [77]:
hip_hop_genres=[]
for song in hip_hop_id:
    hip_hop_genres.append(get_genre(song))

retrying ...1secs
retrying ...1secs
retrying ...1secs


In [78]:
rock_id,rock_names,rock_urls=get_songs_from_playlist_search(limit=10,artist_name='rock')

In [79]:
rock_genres=[]
for song in rock_id:
    rock_genres.append(get_genre(song))

retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [81]:
classical_id,classical_names,classical_urls= get_songs_from_playlist_search(limit=10,artist_name='classical')

In [82]:
classical_genres=[]
for song in classical_id:
    classical_genres.append(get_genre(song))

retrying ...1secs
retrying ...1secs
retrying ...1secs


In [83]:
#make it so there is only one genre
def top_genre(x):
    try:
        return x[0].split(' ')[-1]
    except:
        try:
            return x[0]
        except:
            return 'blank'

In [84]:
techno_clean_genres=[]
pop_clean_genres=[]
grime_clean_genres=[]
garage_clean_genres=[]
hip_hop_clean_genres=[]
rock_clean_genres=[]
classical_clean_genres=[]

In [85]:
all_genres = [
    techno_genres, pop_genres, grime_genres, garage_genres, hip_hop_genres,
    rock_genres,classical_genres
]

all_clean_genres = [
    techno_clean_genres, pop_clean_genres, grime_clean_genres,
    garage_clean_genres, hip_hop_clean_genres, rock_clean_genres,classical_clean_genres
]

In [86]:
#loop through the song genres and get the top genre
for ind, genre in enumerate(all_genres):
    for g in all_genres[ind]:
        all_clean_genres[ind].append(top_genre(g))

In [92]:
all_song_ids = [
    techno_id, pop_id, grime_id, garage_id, hip_hop_id,
    rock_id,classical_id
]

In [93]:
all_names = [
    techno_names, pop_names, grime_names , garage_names, hip_hop_names,
    rock_names, classical_names
]

In [94]:
all_urls =[
    techno_urls, pop_urls, grime_urls, garage_urls, hip_hop_urls,
    rock_urls, classical_urls
]

In [95]:
#zip the song ids and the genres together so can easily filter out genres we dont want
all_zip = [0,1,2,3,4,5,6]
for ind in range(len(all_song_ids)):
    all_zip[ind]= list(zip(all_song_ids[ind],all_names[ind],all_urls[ind],all_clean_genres[ind]))

In [96]:
techno_zip =all_zip[0]
pop_zip = all_zip[1]
grime_zip = all_zip[2]
garage_zip = all_zip[3]
hip_hop_zip =all_zip[4]
rock_zip = all_zip[5]
classical_zip=all_zip[6]

In [98]:
#filter out the genres that you dont want
def filter_by_genre(songs,genres):
    songs_with_genre=[]
    for val in songs:
        if val[-1] in genres:
            songs_with_genre.append(val)
    return songs_with_genre

In [99]:
techno_songs_final= filter_by_genre(techno_zip,['techno','house','electronic'])
pop_songs_final =filter_by_genre(pop_zip,['pop','electropop'])
grime_songs_final =filter_by_genre(grime_zip,['grime','rap'])
garage_songs_final =filter_by_genre(garage_zip,['garage','dancehall','breakbeat','funky'])
hip_hop_songs_final = filter_by_genre(hip_hop_zip,['hop','rap','funk','crunk'])
rock_songs_final= filter_by_genre(rock_zip,['rock','metal''alternative'])
classical_songs_final =filter_by_genre(classical_zip,['classical','performance','baroque','score'])

In [111]:
len(techno_songs_final)+len(pop_songs_final)+len(grime_songs_final)+len(garage_songs_final)+len(hip_hop_songs_final)+len(rock_songs_final)

4952

In [113]:
all_songs_final = [
    techno_songs_final, pop_songs_final, grime_songs_final, garage_songs_final,
    hip_hop_songs_final, rock_songs_final, classical_songs_final
]

In [115]:
#save so dont have to do this process everytime
with open('new_all_songs.pkl', 'wb') as f:
    pickle.dump(all_songs_final,f)

In [4]:
with open('new_all_songs.pkl', 'rb') as f:
     all_songs_final = pickle.load(f)

In [8]:
techno_names=[]
pop_names=[]
hip_hop_names=[]
garage_names=[]
grime_names=[]
rock_names=[]
classical_names=[]

In [9]:
techno_urls=[]
pop_urls=[]
hip_hop_urls=[]
garage_urls=[]
grime_urls=[]
rock_urls=[]
classical_urls=[]

In [10]:
names_list =[
    techno_names, pop_names, hip_hop_names, garage_names,grime_names,rock_names,classical_names
]

url_list = [
    techno_urls, pop_urls, hip_hop_urls, garage_urls, grime_urls, rock_urls,classical_urls
]

In [11]:
for ind, song_list in enumerate(all_songs_final):
    for song in song_list:
        names_list[ind].append(song[1])
        url_list[ind].append(song[2])
        

## Creating Spectograms

In [13]:
techno_specs=[]
pop_specs=[]
hip_specs=[]
garage_specs=[]
grime_specs=[]
rock_specs=[]
classical_specs=[]

techno_names_fin=[]
pop_names_fin=[]
hip_names_fin=[]
garage_names_fin=[]
grime_names_fin=[]
rock_names_fin=[]
classical_names_fin=[]


In [14]:

specs_list = [
    techno_specs, pop_specs, hip_specs, garage_specs, grime_specs, rock_specs,classical_specs
]
names_list_fin = [
    techno_names_fin, pop_names_fin, hip_names_fin, garage_names_fin,
    grime_names_fin, rock_names_fin,classical_names_fin
]

In [15]:
#count to keep track of where we are 
count=0
did_not_convert=0
for ind, urls in enumerate(url_list):
    
    for ind_song ,url in enumerate(urls):
        if url != None:
            try:
                mp3_url = url  
                wav = io.BytesIO()
                with urlopen(mp3_url) as r:
                    r.seek = lambda *args: None  # allow pydub to call seek(0)
                    pydub.AudioSegment.from_file(r).export(wav, "wav")

                wav.seek(0)
                y, sr = librosa.load(wav)

                # mel-scaled power (energy-squared) spectrogram
                mel_spec = librosa.feature.melspectrogram(y, sr=sr,n_mels=128,hop_length=1024,n_fft=2048)
                # Convert to log scale (dB)
                log_mel_spec = librosa.amplitude_to_db(mel_spec, ref=np.max)
                #make dimensions of the array smaller
                log_mel_spec = np.resize(log_mel_spec,(128,644))
                #prints count every 5 spectograms
                if ind_song%50==0:
                    print(count)
                count+=1
                #store into feature array
                specs_list[ind].append(log_mel_spec)
                names_list_fin[ind].append(names_list[ind][ind_song])
            #the except is just so it doesnt break if one of the urls is wrong
            #also gives an indication if some of the above code isnt working
            except:
                print('did not convert')
                did_not_convert+=1

(128, 644)
0
(128, 644)
5
(128, 644)
10
(128, 644)
15
(128, 644)
20
(128, 644)
25
(128, 644)
30
(128, 644)
35
(128, 644)
40
(128, 644)
45
(128, 644)
50
(128, 644)
55
(128, 644)
60
(128, 644)
65
(128, 644)
70
(128, 644)
75
(128, 644)
79
(128, 644)
84
(128, 644)
89
(128, 644)
94
(128, 644)
99
(128, 644)
103
(128, 644)
107
(128, 644)
112
(128, 644)
117
(128, 644)
121
(128, 644)
125
(128, 644)
129
(128, 644)
133
(128, 644)
137
(128, 644)
141
(128, 644)
146
(128, 644)
150
(128, 644)
154
(128, 644)
159
(128, 644)
164
(128, 644)
169
(128, 644)
181
(128, 644)
186
(128, 644)
191
(128, 644)
196
(128, 644)
201
(128, 644)
206
(128, 644)
211
(128, 644)
216
(128, 644)
221
(128, 644)
226
(128, 644)
231
(128, 644)
235
(128, 644)
240
(128, 644)
245
(128, 644)
250
(128, 644)
255
(128, 644)
260
(128, 644)
264
(128, 644)
268
(128, 644)
271
(128, 644)
276
(128, 644)
281
(128, 644)
286
(128, 644)
291
(128, 644)
296
(128, 644)
301
(128, 644)
306
(128, 644)
311
(128, 644)
314
(128, 644)
319
(128, 644)
324
(12

(128, 644)
2650
(128, 644)
2654
(128, 644)
2659
(128, 644)
2664
(128, 644)
2668
(128, 644)
2673
(128, 644)
2678
(128, 644)
2682
(128, 644)
2686
(128, 644)
2691
(128, 644)
2696
(128, 644)
2701
(128, 644)
2705
(128, 644)
2709
(128, 644)
2714
(128, 644)
2719
(128, 644)
2724
(128, 644)
2728
(128, 644)
2732
(128, 644)
2737
(128, 644)
2742
(128, 644)
2747
(128, 644)
2752
(128, 644)
2757
(128, 644)
2762
(128, 644)
2766
(128, 644)
2771
(128, 644)
2776
(128, 644)
2781
(128, 644)
2786
(128, 644)
2791
(128, 644)
2796
(128, 644)
2801
(128, 644)
2805
(128, 644)
2809
(128, 644)
2814
(128, 644)
2818
(128, 644)
2822
(128, 644)
2827
(128, 644)
2836
(128, 644)
2842
(128, 644)
2845
(128, 644)
2847
(128, 644)
2851
(128, 644)
2854
(128, 644)
2856
(128, 644)
2859
(128, 644)
2873
(128, 644)
2878
(128, 644)
2883
(128, 644)
2886
(128, 644)
2891
(128, 644)
2894
(128, 644)
2899
(128, 644)
2902
(128, 644)
2906
(128, 644)
2909
(128, 644)
2915
(128, 644)
2918
(128, 644)
2925
(128, 644)
2929
(128, 644)
2930
(128, 64

In [18]:
#save spectograms because its long to do every time
with open('new_spectros.pkl', 'wb') as f:
    pickle.dump(specs_list,f)

In [19]:
#save names of songs
with open('new_names.pkl','wb') as f:
    pickle.dump(names_list_fin,f)